<h1>Desafio Quantamental Itaú - 2023</h1>
<h3>Lamia - Modelo 02</h3>
<h4>Pairs trading c/ Bitcoin e Ethereum</h4>

Este modelo tem o objetivo de realizar uma estratégia de pairs trading com bitcoin e Ethereum.<br>
O pairs trading é uma estratégia de negociação que envolve a identificação de dois ativos que historicamente se movem juntos, formando um par. Nesse caso, vamos criar um modelo simples de machine learning para pares de negociação usando dados históricos do preço do Bitcoin e de outro ativo, por exemplo, o Ethereum.

In [1]:
pip install yfinance

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


In [25]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt


In [17]:
# Função para carregar os dados
def load_data():
    btc_data = yf.download(tickers='BTC-USD', start='2020-01-01', end='2023-01-01')
    eth_data = yf.download(tickers='ETH-USD', start='2020-01-01', end='2023-01-01')

    # Selecionar apenas a coluna de preço de fechamento para cada ativo
    btc_price = btc_data['Close']
    eth_price = eth_data['Close']

    return btc_price, eth_price

In [4]:
# Função para criar os pares de negociação
def create_pairs(btc_price, eth_price, window_size):
    pairs = []
    for i in range(len(btc_price) - window_size):
        btc_window = btc_price[i:i+window_size]
        eth_window = eth_price[i:i+window_size]
        pairs.append((btc_window, eth_window))
    return pairs

In [5]:
# Função para treinar o modelo
def train_model(pairs):
    X = []
    y = []
    for btc_window, eth_window in pairs:
        # Calcular o spread entre os preços de fechamento de Bitcoin e Ethereum
        spread = btc_window - eth_window
        
        # Definir o target como o preço de fechamento de Bitcoin no próximo período
        target = btc_window.iloc[-1]
        
        X.append(spread.values.reshape(-1, 1))
        y.append(target)

In [6]:
# Função para treinar o modelo
def train_model(pairs):
    X = []
    y = []
    for btc_window, eth_window in pairs:
        # Calcular o spread entre os preços de fechamento de Bitcoin e Ethereum
        spread = btc_window - eth_window
        
        # Definir o target como o preço de fechamento de Bitcoin no próximo período
        target = btc_window.iloc[-1]
        
        X.append(spread.values.reshape(-1, 1))
        y.append(target)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    return model, X_test, y_test

In [7]:
# Função para avaliar o modelo
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mse = np.mean((y_pred - y_test)**2)
    print(f'Mean Squared Error: {mse}')

In [8]:
# Função para visualizar o spread e os sinais de negociação
def visualize_results(btc_price, eth_price, spread, buy_signal, sell_signal):
    plt.figure(figsize=(12, 6))
    
    plt.plot(btc_price.index, btc_price.values, label='Bitcoin Price', color='blue')
    plt.plot(eth_price.index, eth_price.values, label='Ethereum Price', color='green')
    
    plt.twinx()
    plt.plot(btc_price.index, spread, label='Spread', color='red')
    
    plt.plot(btc_price.index, [None]*len(btc_price), label='Buy Signal', marker='^', markersize=10, color='g', linestyle='None')
    plt.plot(btc_price.index, [None]*len(btc_price), label='Sell Signal', marker='v', markersize=10, color='r', linestyle='None')
    
    for i in range(len(buy_signal)):
        plt.plot(buy_signal[i], btc_price[buy_signal[i]], 'g^')
    for i in range(len(sell_signal)):
        plt.plot(sell_signal[i], btc_price[sell_signal[i]], 'rv')
    
    plt.title('Pairs Trading - Bitcoin and Ethereum')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend(loc='upper left')
    plt.show()

In [12]:
def main():
    btc_price, eth_price = load_data()
    window_size = 20
    pairs = create_pairs(btc_price, eth_price, window_size)
    
    model, X_test, y_test = train_model(pairs)
    evaluate_model(model, X_test, y_test)
    
    spread = btc_price - eth_price
    signals = model.predict(spread.values.reshape(-1, 1))
    
    # Definir um limiar para os sinais de negociação
    threshold = 0.5
    
    buy_signal = []
    sell_signal = []
    for i in range(len(signals) - 1):
        if signals[i] < -threshold and signals[i + 1] >= -threshold:
            buy_signal.append(spread.index[i + window_size])
        elif signals[i] > threshold and signals[i + 1] <= threshold:
            sell_signal.append(spread.index[i + window_size])
    
    visualize_results(btc_price, eth_price, spread, buy_signal, sell_signal)

if __name__ == '__main__':
    main()

Failed to get ticker 'BTC-USD' reason: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Max retries exceeded with url: /v8/finance/chart/BTC-USD?range=1d&interval=1d (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:1122)')))


[*********************100%***********************]  1 of 1 completed


1 Failed download:
['BTC-USD']: Exception('%ticker%: No timezone found, symbol may be delisted')
Failed to get ticker 'ETH-USD' reason: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Max retries exceeded with url: /v8/finance/chart/ETH-USD?range=1d&interval=1d (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:1122)')))



[*********************100%***********************]  1 of 1 completed


1 Failed download:
['ETH-USD']: Exception('%ticker%: No timezone found, symbol may be delisted')


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.